In [1]:
import pandas as pd
import numpy as np
from datetime import date
import requests

Code to load data

In [2]:
GECKO_URL = 'http://nletf-gecko-p01:8998/api/v1'
#GECKO_URL = 'http://usomm-goat-conf-p02:8998/api/v1'
ETF_PRICINGS_URL = GECKO_URL + '/etfPricings'
INDEX_PRICINGS_URL = GECKO_URL + '/indexPricings'
ALL_MAPPED_PATH = '/allMapped'
BY_RIC_PATH = '/byRic/'
TODAY =date.today()

def load_json_url(url, trade_date=TODAY):
    response = requests.get(url, params={'tradeDate': trade_date})
    response.raise_for_status()
    return response.json()
def get_data(index):
    url = "http://nletf-gecko-p01:8998/api/v1/indexPricings/byRic/." + index
    data = load_json_url(url)
    return data

In [3]:
index= 'STOXX50E'

In [4]:
#Get the whole data for etf pricing:
url= "http://nletf-gecko-p01:8998/api/v1/indexPricings/byRic/.STOXX50E"
index_pricing= load_json_url(url)
index_pricing

ConnectionError: HTTPConnectionPool(host='nletf-gecko-p01', port=8998): Max retries exceeded with url: /api/v1/indexPricings/byRic/.STOXX50E?tradeDate=2019-09-25 (Caused by NewConnectionError('<urllib3.connection.HTTPConnection object at 0x117e63cc0>: Failed to establish a new connection: [Errno 61] Connection refused'))

In [ ]:
#Get the ESX data frame
def from_json_to_df(output):
    constituents = output.get('constituentPricings')
    symbol = [c.get('preferredListingPricing').get('product').get('symbol') for c in constituents]
    date = [c.get('preferredListingPricing').get('pricingDate') for c in constituents]
    weights = [c.get('weight') for c in constituents]
    currency = [c.get('preferredListingPricing').get('product').get('currency') for c in constituents]
    adjClose = [c.get('preferredListingPricing').get('adjustedClosePrice') for c in constituents]
    ESX_contributions = [c.get('weight')*c.get('preferredListingPricing').get('adjustedClosePrice') for c in constituents]
   

    data = {'symbol': symbol, 'date': date, 'weight': weights, 'currency': currency, 'adjClose': adjClose, 'contribution': ESX_contributions}
    ESX = pd.DataFrame(data)

    return ESX

In [ ]:
currencies_df = pd.DataFrame(index_pricing["currencyRates"])
#index_pricing["currencyRates"]
currencies_df.head()

In [ ]:
def get_currency_rate_to_EUR(currency,output):
    currencies_df = pd.DataFrame(output["currencyRates"])
    data = currencies_df
    if currency == 'EUR':
        rate = 1
    else:
        rate_df = data.loc[(data["fromCurrency"]== currency) & (data["toCurrency"]== "EUR"),"rate"].reset_index()
        rate= rate_df.loc[0,"rate"]
    return rate

In [ ]:
def get_divData(index,today,maturity):
    url_div = "http://nletf-gecko-p01:8998/api/v1/indexDividends/byRic/." + index + "?minExDivDate="+ str(today) +"&maxExDivDate=" + str(maturity)
    div_data = load_json_url(url_div)
    
    return div_data
    

In [ ]:
def from_json_to_df_div(output):
    constituentDividends = output.get('constituentDividends')
    symbol = [cd.get("product").get("symbol") for cd in constituentDividends]
    currency = [cd.get("dividend").get("currency") for cd in constituentDividends]
    exDividend = [cd.get("dividend").get('exDivDate') for cd in constituentDividends]
    grossAmount = [cd.get("dividend").get('grossAmount') for cd in constituentDividends]
    weight= [cd.get('weight') for cd in constituentDividends]
    type = [cd.get("dividend").get('type') for cd in constituentDividends]
    data = {'symbol': symbol,'exDividend': exDividend, 'grossAmount': grossAmount, 'weight': weight,"currency": currency,"type": type}
    data_div = pd.DataFrame(data)
    return data_div


In [ ]:
today =date.today()
maturity= date(2019,12,21)

out_put = get_divData(index,TODAY,MATURITY)
out_put.keys()

In [ ]:
TODAY =date.today()
MATURITY= date(2019,12,20)
def get_divIndex(index,today= TODAY,maturity = MATURITY):
    out_put = get_divData(index,today,maturity)
    div_df= from_json_to_df_div(out_put)
    
    for i in range(len(div_df.currency)):
        div_df.loc[i,"currencyRate"]= get_currency_rate_to_EUR(div_df.loc[i,"currency"],out_put)
        #print(get_currency_rate_to_EUR(div_df.loc[i,"currency"],out_put))
    div_df["adjustedDiv"]= div_df["grossAmount"] * div_df["weight"] * div_df["currencyRate"]
    div_df.loc[div_df["type"] == "SPEC","adjustedDiv"] =0
    DVD  = sum(div_df["adjustedDiv"])
    
    return DVD
    

In [ ]:
def get_efp(spot,div,interestRate,ssr,maturity):
    today =date.today()
    t= ((maturity - today).days)/365
    efp = (interestRate -ssr)* spot*t - div 
    return efp

In [ ]:
DVD = get_divIndex(index)
index_close= get_data(index).get('closePrice')
interestRate = -0.0016588592319718195
ssr = 0.0025
EFP= get_efp(index_close,DVD,interestRate,ssr,maturity)
EFP

In [ ]:
efp1 = -15.25
today =date.today()
t= ((maturity - today).days)/365
i = (efp1 +DVD + ssr*index_close*t)/(index_close*t)
i

Exercise about checking ETF

In [ ]:
url = "http://nletf-gecko-p01:8998/api/v1/etfPricings/byRic/C5S.PA?"
output1 = load_json_url(url)


In [ ]:
output1.get('')

In [ ]:
nav_dict = {
    'closePrice' :[output1.get('closePrice').get('closePrice')], \
    'open' :[output1.get('open')], \
    'issuerTradeDate' :[output1.get('issuerTradeDate')], \
    'issuerNav' :[output1.get('issuerNav')], \
    'impliedNav' :[output1.get('impliedNav')], \
    'impliedIssuerNav' :[output1.get('impliedIssuerNav')], \
    'preferredNav' :[output1.get('preferredNav')], \
    'calculationCash' :[output1.get('calculationCash')], \
        'pricingDate' :[output1.get('pricingDate')], \
}


In [ ]:
nav_df = pd.DataFrame(nav_dict)
nav_df

In [ ]:
output1.get('closePrice').get('closePrice')

In [ ]:
x = output1.get('preferredBasketPricing').get('constituentPricings')
x.keys()

In [ ]:
def from_json_to_data_frame_preferred_pricing(output):
    #Issuer data
    constituents = output.get('preferredBasketPricing').get('constituentPricings')
    pricing_date = [c.get('preferredListingPricing').get('pricingDate') for c in constituents]
    symbol = [c.get('preferredListingPricing').get('product').get('symbol') for c in constituents]
    #date = [c.get('preferredListingPricing').get('pricingDate') for c in constituents]
    preferWeight = [c.get('weight') for c in constituents]
    prefercurrency = [c.get('preferredListingPricing').get('currency') for c in constituents]
    preferAdjustedClosePrice = [c.get('preferredListingPricing').get('adjustedClosePrice') for c in constituents]
    preferContribution = [c.get('preferredListingPricing').get('contribution') for c in constituents]
    

    data = {'pricing_date': pricing_date, \
            'symbol': symbol, \
            'preferWeights': preferWeight, \
            'prefercurrency': prefercurrency, \
            'preferAdjustedClosePrice': preferAdjustedClosePrice, \
            'preferContribution': preferContribution \
           }
    data_df = pd.DataFrame(data)
    return data_df

In [ ]:
preferData_df = from_json_to_data_frame_preferred_pricing(output1)
preferData_df['weight*adjustedPrice'] = preferData_df['preferWeights']*preferData_df['preferAdjustedClosePrice']
preferEtfPrice= sum(preferData_df['weight*adjustedPrice']) + output1.get('calculationCash')

In [ ]:
nav_df.loc[0,'issuerNav']- nav_df.loc[0,'calculationCash']

In [ ]:
preferEtfPrice

Check with yesterday data